In [ ]:
import re
import json
from pathlib import Path
from pprint import pprint
from datasets import load_dataset, DatasetDict
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
def push_to_ourhub(
    dsd: DatasetDict,
    dataset_name: str,
    dataset_path: str = "appier-ai-research/StreamBench"
) -> None:
    dsd.push_to_hub(dataset_path, dataset_name)

load_path = "hotpot_qa"
dataset_name = "distractor"
dsd = load_dataset(load_path, dataset_name)
# push_to_ourhub(dsd, dataset_name="hotpotqa_distract")
dsd

In [ ]:
dsd = DatasetDict({
    "test": dsd["validation"].select(range(1500))
})

In [ ]:
push_to_ourhub(dsd, dataset_name="hotpotqa_distract")

## DS-1000

In [ ]:
ds = load_dataset("xlangai/DS-1000")

In [ ]:
row = ds["test"][0]
row.keys()

In [ ]:
print(row["reference_code"])

In [ ]:
print(row["prompt"])

In [ ]:
print(row["code_context"])

In [ ]:
def extract_after_exec_content(code_context: str) -> str:
    return re.findall(pattern=r'exec_context.*"""', string=code_context, flags=re.DOTALL)

tests = list()
for row in ds["test"]:
    test = extract_after_exec_content(code_context=row["code_context"])
    tests.append(test)

## ToolBench

In [ ]:
with open("../../../simulated-trial-and-error/llama-recipes/ft_datasets/tool_test.json") as f:
    dataset = json.load(f)

API_name_list = list(dataset.keys())

with open("../../../simulated-trial-and-error/STE/tool_metadata/API_descriptions.json") as f:
    api_desc = json.load(f)

apiname2desc = {apiname: api_desc[apiname] for apiname in API_name_list}
print(json.dumps(apiname2desc, indent=4))

In [ ]:
dataset = load_dataset("appier-ai-research/toolbench-50apis", "")

In [ ]:
dataset["test"][0]

# Classification

## ANLI

In [ ]:
DATASET_PATH = "facebook/anli"
DATASET_NAME = ""
anli = load_dataset(DATASET_PATH, DATASET_NAME)

In [ ]:
pprint(anli.keys())
pprint(anli["train_r1"])

In [ ]:
anli["train_r1"][7071]

## DDXPlus

In [ ]:
# TODO: Option 1: see whether DDXPlus has official preprocessing scripts
# TODO: Option 2: make a pypi package for preprocessing DDXPlus

## MultiFin

In [ ]:
DATASET_PATH = "TheFinAI/flare-multifin-en"
DATASET_NAME = ""
multifin_en = load_dataset(DATASET_PATH, DATASET_NAME)

In [ ]:
dataset = multifin_en["test"]

In [ ]:
dataset_dict = dataset.train_test_split(train_size=16 / len(dataset), shuffle=True, seed=42)

In [ ]:
DATASET_PATH = "TheFinAI/flare-es-multifin"
DATASET_NAME = ""
multifin_es = load_dataset(DATASET_PATH, DATASET_NAME)

In [ ]:
pprint(multifin_es.keys())
pprint(multifin_es["test"][12])
s = multifin_es["test"][0]["query"]

## Twitter Eval

In [ ]:
DATASET_PATH = "tweet_eval"
DATASET_NAME = "sentiment"

dataset = load_dataset(DATASET_PATH, DATASET_NAME)

In [ ]:
dataset["validation"][2]["text"]

In [ ]:
dataset["train"]

## SST5

## MMLU

In [ ]:
DATASET_PATH = "hails/mmlu_no_train"
DATASET_NAME = "all"
mmlu = load_dataset(DATASET_PATH, DATASET_NAME)

In [ ]:
mmlu["test"][0]

In [ ]:
from tqdm import tqdm
pprint(mmlu.keys())

answer_cnt = dict()
choice_lens = set()
for split in mmlu.keys():
    for row in tqdm(mmlu[split]):
        choice_lens.add(len(row["choices"]))
        answer = row["answer"]
        if answer in answer_cnt:
            answer_cnt[answer] += 1
        else:
            answer_cnt[answer] = 1

In [ ]:
mmlu["dev"][0]

## CMMLU

In [ ]:
task_list = ['agronomy', 'anatomy', 'ancient_chinese', 'arts', 'astronomy', 'business_ethics', 'chinese_civil_service_exam', 'chinese_driving_rule', 'chinese_food_culture', 'chinese_foreign_policy', 'chinese_history', 'chinese_literature', 
'chinese_teacher_qualification', 'clinical_knowledge', 'college_actuarial_science', 'college_education', 'college_engineering_hydrology', 'college_law', 'college_mathematics', 'college_medical_statistics', 'college_medicine', 'computer_science',
'computer_security', 'conceptual_physics', 'construction_project_management', 'economics', 'education', 'electrical_engineering', 'elementary_chinese', 'elementary_commonsense', 'elementary_information_and_technology', 'elementary_mathematics', 
'ethnology', 'food_science', 'genetics', 'global_facts', 'high_school_biology', 'high_school_chemistry', 'high_school_geography', 'high_school_mathematics', 'high_school_physics', 'high_school_politics', 'human_sexuality',
'international_law', 'journalism', 'jurisprudence', 'legal_and_moral_basis', 'logical', 'machine_learning', 'management', 'marketing', 'marxist_theory', 'modern_chinese', 'nutrition', 'philosophy', 'professional_accounting', 'professional_law', 
'professional_medicine', 'professional_psychology', 'public_relations', 'security_study', 'sociology', 'sports_science', 'traditional_chinese_medicine', 'virology', 'world_history', 'world_religions']

cmmlu = {k: load_dataset(r"haonan-li/cmmlu", k) for k in task_list}

In [ ]:
from datasets import concatenate_datasets

def merge_sets(datasets: dict) -> dict:
    print("Merging datasets...")
    test_sets = list()
    dev_sets = list()
    for subject, dataset in datasets.items():
        test_set = dataset["test"].map(lambda row: {**row, "subject": subject}, )
        dev_set = dataset["dev"].map(lambda row: {**row, "subject": subject})
        test_sets.append(test_set)
        dev_sets.append(dev_set)

    merged_test = concatenate_datasets(test_sets)
    merged_dev = concatenate_datasets(dev_sets)
    return {"test": merged_test, "dev": merged_dev}

merged = merge_sets(cmmlu)

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from datasets import DatasetDict

cmmlu_all = DatasetDict(merged)

In [ ]:
cmmlu_all.push_to_hub("appier-ai-research/cmmlu_all", private=True)

In [ ]:
cmmlu_all2 = load_dataset("appier-ai-research/cmmlu_all")

In [ ]:
pprint(cmmlu.keys())
pprint(cmmlu["agronomy"]["test"][0])

## TMMLU+

In [ ]:
DATASET_PATH = "appier-ai-research/tmmluplus_timeline"
task_list = [
    'engineering_math',
    # 'dentistry', 'traditional_chinese_medicine_clinical_medicine', 'clinical_psychology', 'technical', 'culinary_skills', 'mechanical', 'logic_reasoning', 'real_estate',
    # 'general_principles_of_law', 'finance_banking', 'anti_money_laundering', 'ttqav2', 'marketing_management', 'business_management', 'organic_chemistry', 'advance_chemistry',
    # 'physics', 'secondary_physics', 'human_behavior', 'national_protection', 'jce_humanities', 'politic_science', 'agriculture', 'official_document_management',
    # 'financial_analysis', 'pharmacy', 'educational_psychology', 'statistics_and_machine_learning', 'management_accounting', 'introduction_to_law', 'computer_science', 'veterinary_pathology',
    # 'accounting', 'fire_science', 'optometry', 'insurance_studies', 'pharmacology', 'taxation', 'trust_practice', 'geography_of_taiwan', 'physical_education', 'auditing', 'administrative_law',
    # 'education_(profession_level)', 'economics', 'veterinary_pharmacology', 'nautical_science', 'occupational_therapy_for_psychological_disorders',
    # 'basic_medical_science', 'macroeconomics', 'trade', 'chinese_language_and_literature', 'tve_design', 'junior_science_exam', 'junior_math_exam', 'junior_chinese_exam',
    # 'junior_social_studies', 'tve_mathematics', 'tve_chinese_language', 'tve_natural_sciences', 'junior_chemistry', 'music', 'education', 'three_principles_of_people',
    # 'taiwanese_hokkien'
]
splits = ["train", "validation", "test"]

tmmluplus = {task: load_dataset(DATASET_PATH, task) for task in task_list}

In [ ]:
tmmluplus["engineering_math"]["test"][0]

In [ ]:
pprint(tmmluplus.keys())
print(tmmluplus["engineering_math"]["train"][0])

In [ ]:
tmmluplus = load_dataset("appier-ai-research/tmmluplus_timeline_aggregated")

In [ ]:
d = dict()  # subject: count

split = "train"
for row in tmmluplus[split]:
    d[row["name"]] = d.get(row["name"], 0) + 1

d

## MedQA

In [ ]:
DATASET_PATH = "bigbio/med_qa"
subsets = ["med_qa_en_4options_source", "med_qa_zh_4options_source", "med_qa_tw_source"]
splits = {"train", "validation", "test"}

medqa = {subset: load_dataset(DATASET_PATH, subset) for subset in subsets}

In [ ]:
medqa

In [ ]:
s[4]

In [ ]:
pprint(medqa.keys())
pprint(medqa["med_qa_en_4options_source"]["validation"][0])
pprint(medqa["med_qa_zh_4options_source"]["validation"][0])
pprint(medqa["med_qa_tw_source"]["validation"][0])

## Playground

In [ ]:
from enum import Enum

class RetrieveOrder(Enum):
    SIMILAR_AT_TOP = "similar_at_top"  # the most similar retrieved chunk is ordered at the top
    SIMILAR_AT_BOTTOM = "similar_at_bottom"  # reversed

In [ ]:
member_values = {member.value for member in RetrieveOrder}

member_values